In [1]:
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import face_recognition

In [2]:
train_sample_metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [3]:
train_sample = '../input/deepfake-detection-challenge/train_sample_videos/'
train_video_files = [train_sample + x for x in sorted(os.listdir(train_sample))]

In [4]:
len(train_video_files)

401

In [5]:
sorted(train_video_files)

['../input/deepfake-detection-challenge/train_sample_videos/aagfhgtpmv.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/aapnvogymq.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abarnvbtwb.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abofeumbvv.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abqwwspghj.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acifjvzvpm.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acqfdwsrhi.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acxnxvbsxk.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acxwigylke.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/aczrgyricp.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adhsbajydo.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adohikbdaz.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adylbeequz.mp4',

In [6]:
# original이 있는 FAKE
train_sample_metadata['FAKE has original'] = [(train_sample_metadata.iloc[:,2][i] in list(train_sample_metadata.index)) for i in train_sample_metadata.T]

,label,split,original,FAKE has original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4,False
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4,False
abarnvbtwb.mp4,REAL,train,None,False
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4,True
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4,False
...,...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4,False
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4,True
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4,True
eudeqjhdfd.mp4,REAL,train,None,False


In [17]:
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'])

58

In [16]:
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique())

42

In [18]:
train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique()

array(['atvmxvwyns.mp4', 'ccfoszqabv.mp4', 'dlpoieqvfb.mp4',
       'aytzyidmgs.mp4', 'dkuayagnmc.mp4', 'bzythlfnhq.mp4',
       'cppdvdejkc.mp4', 'bulkxhhknf.mp4', 'edyncaijwx.mp4',
       'dzyuwjkjui.mp4', 'efwfxwwlbw.mp4', 'cprhtltsjp.mp4',
       'djxdyjopjd.mp4', 'cpjxareypw.mp4', 'dbtbbhakdv.mp4',
       'ellavthztb.mp4', 'bxzakyopjf.mp4', 'cizlkenljw.mp4',
       'brwrlczjvi.mp4', 'drcyabprvt.mp4', 'crezycjqyk.mp4',
       'cyxlcuyznd.mp4', 'ekcrtigpab.mp4', 'ehtdtkmmli.mp4',
       'bwipwzzxxu.mp4', 'atkdltyyen.mp4', 'chtapglbcj.mp4',
       'caifxvsozs.mp4', 'bffwsjxghk.mp4', 'duycddgtrl.mp4',
       'ehccixxzoe.mp4', 'eckvhdusax.mp4', 'dakiztgtnw.mp4',
       'bgwmmujlmc.mp4', 'abarnvbtwb.mp4', 'bejhvclboh.mp4',
       'avmjormvsx.mp4', 'egghxjjmfg.mp4', 'dbnygxtwek.mp4',
       'cmbzllswnl.mp4', 'afoovlsmtx.mp4', 'bdnaqemxmr.mp4'], dtype=object)

In [19]:
original_vidio_has_fake = ['../input/deepfake-detection-challenge/train_sample_videos/'+ i for i in train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique()]

['../input/deepfake-detection-challenge/train_sample_videos/atvmxvwyns.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/ccfoszqabv.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/dlpoieqvfb.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/aytzyidmgs.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/dkuayagnmc.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/bzythlfnhq.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/cppdvdejkc.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/bulkxhhknf.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/edyncaijwx.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/dzyuwjkjui.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/efwfxwwlbw.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/cprhtltsjp.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/djxdyjopjd.mp4',

In [5]:
train_sample_metadata = pd.read_json(os.path.join(train_sample, 'metadata.json')).T
train_video_files = sorted(train_video_files)

detector = dlib.get_frontal_face_detector()
vid_num = 0
all_vid = len(train_video_files)
for vid in train_video_files:
    count = 0
    frame = 0
    file_name_mp4 = vid.split('/')[-1]
    file_name = file_name_mp4.split('.')[0]
    label = train_sample_metadata.loc[file_name_mp4][0]
    origin = train_sample_metadata.loc[file_name_mp4][2]
    if not origin:
        origin = 'NONE'
    else:
        origin = origin.split('.')[0]

    cap = cv2.VideoCapture(vid)
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        if frame % 30 == 0:
            ret, image = cap.retrieve()
            face_rects, scores, idx = detector.run(image, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                li = [x1,y1,x2,y2]
                li = [0 if i<0 else li[idx] for idx, i in enumerate(li)]
                x1,y1,x2,y2 = li

                landmarks = face_recognition.face_landmarks(crop_img)
                pil = Image.fromarray(crop_img)
                p = ImageDraw.Draw(pil)
                for idx, landmark in enumerate(landmarks):
                    for mark in landmark.keys():
                        p.line(landmark[mark], width=3)
                cv2.imwrite(f'../working/dataset/{label}/{file_name}_{count}_{label}_{origin}.jpg', cv2.resize(np.array(pil), (128, 128)))
                print(f'../working/dataset/{label}/{file_name}_{count}_{label}_{origin}.jpg')
                count += 1
        frame += 1
    print(f'{vid_num}/{all_vid}')
    vid_num += 1
    cap.release()

dataset/FAKE/aagfhgtpmv_0_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_1_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_2_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_3_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_4_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_5_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_6_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_7_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_8_FAKE_vudstovrck.jpg
dataset/FAKE/aagfhgtpmv_9_FAKE_vudstovrck.jpg
0/400
dataset/FAKE/aapnvogymq_0_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_1_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_2_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_3_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_4_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_5_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_6_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_7_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_8_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_9_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_10_FAKE_jdubbvfswz.jpg
dataset/FAKE/aapnvogymq_11_

dataset/FAKE/agqphdxmwt_7_FAKE_aytzyidmgs.jpg
19/400
dataset/REAL/agrmhtjdlk_0_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_1_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_2_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_3_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_4_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_5_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_6_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_7_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_8_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_9_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_10_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_11_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_12_REAL_NONE.jpg
dataset/REAL/agrmhtjdlk_13_REAL_NONE.jpg
20/400
dataset/FAKE/ahbweevwpv_0_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_1_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_2_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_3_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_4_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_5_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_6_FAKE_dkuayagnmc.jpg
dataset/FAKE/ahbweevwpv_7_FAKE_dkuayagnmc.jpg
dataset/

dataset/FAKE/altziddtxi_9_FAKE_meawmsgiti.jpg
37/400
dataset/FAKE/alvgwypubw_0_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_1_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_2_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_3_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_4_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_5_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_6_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_7_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_8_FAKE_vcxckqbaya.jpg
dataset/FAKE/alvgwypubw_9_FAKE_vcxckqbaya.jpg
38/400
dataset/FAKE/amaivqofda_0_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_1_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_2_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_3_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_4_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_5_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_6_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_7_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_8_FAKE_xobhsemxmv.jpg
dataset/FAKE/amaivqofda_9_FAKE_xobhsemxmv.jpg
39/400
dataset/FAKE/

dataset/REAL/atkdltyyen_7_REAL_NONE.jpg
dataset/REAL/atkdltyyen_8_REAL_NONE.jpg
dataset/REAL/atkdltyyen_9_REAL_NONE.jpg
57/400
58/400
dataset/FAKE/atxvxouljq_0_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_1_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_2_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_3_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_4_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_5_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_6_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_7_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_8_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_9_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_10_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_11_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_12_FAKE_wwqiuiwdbz.jpg
dataset/FAKE/atxvxouljq_13_FAKE_wwqiuiwdbz.jpg
59/400
dataset/FAKE/atyntldecu_0_FAKE_mfzqxktxud.jpg
dataset/FAKE/atyntldecu_1_FAKE_mfzqxktxud.jpg
dataset/FAKE/atyntldecu_2_FAKE_mfzqxktxud.jpg
dataset/FAKE/atyntldecu_3_FAKE_mfzqxktxud.jpg
dataset/FAKE/atyntldecu_4_F

dataset/FAKE/axczxisdtb_7_FAKE_meawmsgiti.jpg
dataset/FAKE/axczxisdtb_8_FAKE_meawmsgiti.jpg
dataset/FAKE/axczxisdtb_9_FAKE_meawmsgiti.jpg
76/400
dataset/REAL/axntxmycwd_0_REAL_NONE.jpg
dataset/REAL/axntxmycwd_1_REAL_NONE.jpg
dataset/REAL/axntxmycwd_2_REAL_NONE.jpg
dataset/REAL/axntxmycwd_3_REAL_NONE.jpg
dataset/REAL/axntxmycwd_4_REAL_NONE.jpg
dataset/REAL/axntxmycwd_5_REAL_NONE.jpg
77/400
dataset/FAKE/axoygtekut_0_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_1_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_2_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_3_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_4_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_5_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_6_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_7_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_8_FAKE_topyiohccg.jpg
dataset/FAKE/axoygtekut_9_FAKE_topyiohccg.jpg
78/400
79/400
dataset/FAKE/axwovszumc_0_FAKE_rvoudrbyac.jpg
dataset/FAKE/axwovszumc_1_FAKE_rvoudrbyac.jpg
dataset/FAKE/axwovszumc_2_FAKE_rvoudrbyac.

...

dataset/FAKE/dqzreruvje_6_FAKE_rlldzrnmdn.jpg
dataset/FAKE/dqzreruvje_7_FAKE_rlldzrnmdn.jpg
dataset/FAKE/dqzreruvje_8_FAKE_rlldzrnmdn.jpg
dataset/FAKE/dqzreruvje_9_FAKE_rlldzrnmdn.jpg
305/400
dataset/REAL/drcyabprvt_0_REAL_NONE.jpg
dataset/REAL/drcyabprvt_1_REAL_NONE.jpg
dataset/REAL/drcyabprvt_2_REAL_NONE.jpg
dataset/REAL/drcyabprvt_3_REAL_NONE.jpg
dataset/REAL/drcyabprvt_4_REAL_NONE.jpg
dataset/REAL/drcyabprvt_5_REAL_NONE.jpg
dataset/REAL/drcyabprvt_6_REAL_NONE.jpg
dataset/REAL/drcyabprvt_7_REAL_NONE.jpg
dataset/REAL/drcyabprvt_8_REAL_NONE.jpg
dataset/REAL/drcyabprvt_9_REAL_NONE.jpg
306/400
dataset/FAKE/drgjzlxzxj_0_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_1_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_2_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_3_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_4_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_5_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_6_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_7_FAKE_yagllixjvh.jpg
dataset/FAKE/drgjzlxzxj_8_FAKE_y

dataset/REAL/dxbqjxrhin_1_REAL_NONE.jpg
dataset/REAL/dxbqjxrhin_2_REAL_NONE.jpg
dataset/REAL/dxbqjxrhin_3_REAL_NONE.jpg
dataset/REAL/dxbqjxrhin_4_REAL_NONE.jpg
dataset/REAL/dxbqjxrhin_5_REAL_NONE.jpg
dataset/REAL/dxbqjxrhin_6_REAL_NONE.jpg
324/400
dataset/FAKE/dxuliowugt_0_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_1_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_2_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_3_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_4_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_5_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_6_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_7_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_8_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_9_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_10_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_11_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_12_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_13_FAKE_imzqmbfugn.jpg
dataset/FAKE/dxuliowugt_14_FAKE_imzqmbfugn.jpg
325/400
dataset/FAKE/dxuplhwvig_0_FAKE_yoavqsqobz.jpg
dat

dataset/FAKE/ecwaxgutkc_8_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_9_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_10_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_11_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_12_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_13_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_14_FAKE_itmwoyxbas.jpg
dataset/FAKE/ecwaxgutkc_15_FAKE_itmwoyxbas.jpg
343/400
dataset/FAKE/eczrseixwq_0_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_1_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_2_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_3_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_4_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_5_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_6_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_7_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_8_FAKE_iieoqptzec.jpg
dataset/FAKE/eczrseixwq_9_FAKE_iieoqptzec.jpg
344/400
dataset/REAL/edyncaijwx_0_REAL_NONE.jpg
dataset/REAL/edyncaijwx_1_REAL_NONE.jpg
dataset/REAL/edyncaijwx_2_REAL_NONE.jpg
dataset/REAL/edyncaijwx_3_REAL

dataset/REAL/ehtdtkmmli_7_REAL_NONE.jpg
dataset/REAL/ehtdtkmmli_8_REAL_NONE.jpg
dataset/REAL/ehtdtkmmli_9_REAL_NONE.jpg
363/400
dataset/FAKE/eiriyukqqy_0_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_1_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_2_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_3_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_4_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_5_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_6_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_7_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_8_FAKE_ixuouyigxa.jpg
dataset/FAKE/eiriyukqqy_9_FAKE_ixuouyigxa.jpg
364/400
dataset/FAKE/eivxffliio_0_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_1_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_2_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_3_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_4_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_5_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_6_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_7_FAKE_joeifeskbs.jpg
dataset/FAKE/eivxffliio_8_FAKE_joeif

dataset/REAL/eqnoqyfquo_3_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_4_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_5_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_6_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_7_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_8_REAL_NONE.jpg
dataset/REAL/eqnoqyfquo_9_REAL_NONE.jpg
383/400
dataset/FAKE/eqvuznuwsa_0_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_1_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_2_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_3_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_4_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_5_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_6_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_7_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_8_FAKE_cmbzllswnl.jpg
dataset/FAKE/eqvuznuwsa_9_FAKE_cmbzllswnl.jpg
384/400
dataset/REAL/erlvuvjsjf_0_REAL_NONE.jpg
dataset/REAL/erlvuvjsjf_1_REAL_NONE.jpg
dataset/REAL/erlvuvjsjf_2_REAL_NONE.jpg
dataset/REAL/erlvuvjsjf_3_REAL_NONE.jpg
dataset/REAL/erlvuvjsjf_4_REAL_NONE.jpg
dataset/REAL/erlvuvjsjf_5_REAL_NONE.jpg
data